# Quick CIGALE Notebook HOW TO

### First we import all the necessary things. Make sure you have the FRB repo

In [22]:
# importing cigale wrapper
from cigale import run

In [20]:
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.table import Table
import numpy as np
import os

from frb.galaxies import defs
from frb.surveys import survey_utils as su
from frb.galaxies.photom import correct_photom_table
from frb.galaxies.nebular import get_ebv
from frb.surveys.catalog_utils import convert_mags_to_flux

In [15]:
# set NEDLVS env variable
os.environ['NEDLVS'] = "/home/lordrick/Projects/fun_side_projects/MODaP_tests/nedlvs_file.fits" # Replace with your own file path to the nedlvs file

In [8]:
# Example usage
gal_name = "J060740.40+433447.57"
gal_z = 0.2411

In [9]:
# Get coordinates
coord = SkyCoord.from_name(gal_name)

In [11]:
def get_public_photometry(gal_name, gal_coord, radius=1*u.arcsec):
    """
    Obtain public photometry for the host galaxy. This currently relies
    on the search_all_surveys function in frb.surveys.survey_utils.
    """
    
    # Get the photometry from all known surveys
    phot_tbl = su.search_all_surveys(gal_coord, radius, include_radio=False)

    if len(phot_tbl) > 0:
        # sort by separation and assign the closest if multiple
        phot_tbl.sort('separation')
        phot_tbl['Name'] = 'HGXXXXXXXXX' # placeholder when there are multiple objects
        phot_tbl[0]['Name'] = gal_name # assign name to the closest object
        ebv = get_ebv(gal_coord)['meanValue']
        correct_photom_table(phot_tbl,ebv,name=gal_name)
        photom = parse_photom(gal_coord, Table(phot_tbl[0]))
    else:
        print("No public photometry found for the host galaxy.")
    return photom

def parse_photom(coord, phot_tbl, max_off=1*u.arcsec, 
                    overwrite=True, EBV=None):
    """
    Parse photometry from an input table

    Fills the photom dict

    Also fills fluxes in mJy

    Args:
        phot_tbl (astropy.table.Table):
            ra, dec entires are required
        max_off (Angle, optional):
        overwrite (bool, optional):
        EBV (float, optional):  Galactic reddening.  If included, the photometry
            has been corrected for this.  If not, who knows?!  :)

    Returns:

    """
    phot_coord = SkyCoord(ra=phot_tbl['ra'], dec=phot_tbl['dec'], unit='deg')
    sep = coord.separation(phot_coord)
    row = np.argmin(sep)
    # Satisfy minimum offset?
    if sep[row] > max_off:
        print("No photometric sources within {} of the galaxy".format(max_off))
        return
    # Get a flux table
    flux_tbl = convert_mags_to_flux(phot_tbl, fluxunits='mJy')


    photom = {}

    # Fill fluxes (mJy)
    for filter in defs.valid_filters:
        # Value
        if filter in phot_tbl.keys():
            if (filter in photom.keys()) and (not overwrite):
                pass
            else:
                # -999. is used as empty fill value
                if phot_tbl[filter][row] < -990:
                    pass
                else:
                    photom[filter] = phot_tbl[filter][row]
                    photom[filter+'_flux'] = flux_tbl[filter][row]
                    # Try error
                    if filter+'_err' in phot_tbl.keys():
                        photom[filter+'_err'] = phot_tbl[filter+'_err'][row]
                        photom[filter+'_flux_err'] = flux_tbl[filter+'_err'][row]
                    # Try reference
                    if filter+'_ref' in phot_tbl.keys():
                        photom[filter+'_ref'] = phot_tbl[filter+'_ref'][row]


                    # Add entries for corresponding flux values.
    # EBV
    if EBV is not None:
        photom['EBV'] = EBV

    return photom

In [16]:
# Get public photometry
photom = get_public_photometry(gal_name, coord)

/home/lordrick/Projects/F4_Repos/FRB/frb/surveys/panstarrs.py:165: UserWarning: mastcasjobs not installed. Cannot download photometric redshifts.
  warnings.warn("mastcasjobs not installed. Cannot download photometric redshifts.")


Found 1 objects in Pan-STARRS
Found 1 objects in WISE
Empty table in SDSS
Empty table in DES
Empty table in DESI
Empty table in DELVE
Empty table in DECaL
Empty table in VISTA
Empty table in NSC


/home/lordrick/Projects/F4_Repos/FRB/frb/surveys/survey_utils.py:219: RuntimeWarning: Couldn't connect to HSC. Skipping this for now.
  warnings.warn("Couldn't connect to {:s}. Skipping this for now.".format(surveyname), RuntimeWarning)


Empty table in NEDLVS
Empty table in 2MASS
Empty table in GALEX


/home/lordrick/miniconda3/envs/frb_env/lib/python3.12/site-packages/astropy/table/row.py:71: StringTruncateWarning: truncated right side string(s) longer than 11 character(s) during assignment
  self._table.columns[item][self._index] = val


No matches to input name=J060740.40+433447.57.  Returning


### Run CIGALE

In [23]:
def run_cigale(photom,gal_z,gal_name, data_file="cigale_in.fits", config_file="pcigale.ini",
    wait_for_input=False, save_sed=True, plot=True, outdir='out',**kwargs):
    """
    Generates the input data file for CIGALE
    given the photometric points and redshift
    of a galaxy

    Args:
        ID: str, optional
            An ID for the galaxy. If none, "GalaxyA" is assigned.
        data_file (str, optional):
            Root name for the photometry data file generated used as input to CIGALE
        config_file (str, optional):
            Root name for the file where CIGALE's configuration is generated
        wait_for_input (bool, optional):
            If true, waits for the user to finish editing the auto-generated config file
            before running.
        save_sed (bool, optional):
            Saves the best fit SED if true
        plot (bool, optional):
            Plots the best fit SED if true
        cores (int, optional):
            Number of CPU cores to be used. Defaults
            to all cores on the system.
        outdir (str, optional):
            Path to the many outputs of CIGALE
            If not supplied, the outputs will appear in a folder named out/

    kwargs:  These are passed into gen_cigale_in() and _initialise()
        sed_modules (list of 'str', optional):
            A list of SED modules to be used in the 
            PDF analysis. If this is being input, there
            should be a corresponding correct dict
            for sed_modules_params.
        sed_module_params (dict, optional):
            A dict containing parameter values for
            the input SED modules. Better not use this
            unless you know exactly what you're doing.
    """
    

    new_photom = Table([photom])
    new_photom['redshift'] = gal_z
    if gal_name != '':
        new_photom['ID'] = gal_name
    else:
        new_photom['ID'] = 'Galaxy'


    run(photometry_table = new_photom,
        zcol = 'redshift',
        data_file = data_file,
        config_file = config_file,
        wait_for_input = wait_for_input,
        save_sed = save_sed,
        plot = plot,
        outdir = outdir,
        **kwargs)
    
    return 

In [24]:
run_cigale(photom,gal_z,gal_name)

[INFO] Initialising the analysis module.

──────────────────────────────────────────────────── Block 1/1 ────────────────────────────────────────────────────

[INFO] Computing models.

[INFO] Done.

[INFO] Estimating the physical properties.

...

[INFO] Sanity check of the analysis results.

[INFO] 0.0% of the objects have χ²_red~0 and 0.0% χ²_red<0.5.

[INFO] Saving the analysis results.

[INFO] Run completed! 👍

[INFO] Done.

## Save values to .json file